In [162]:
--Selected all records within the dataset
--Ordered by number of games sold highest to lowest
--Top ten only
Select
    *
from
    game_sales
order by
    games_sold desc
limit 10;

10 rows affected.


game,platform,publisher,developer,games_sold,year
Wii Sports for Wii,Wii,Nintendo,Nintendo EAD,82.90,2006
Super Mario Bros. for NES,NES,Nintendo,Nintendo EAD,40.24,1985
Counter-Strike: Global Offensive for PC,PC,Valve,Valve Corporation,40.00,2012
Mario Kart Wii for Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
PLAYERUNKNOWN'S BATTLEGROUNDS for PC,PC,PUBG Corporation,PUBG Corporation,36.60,2017
Minecraft for PC,PC,Mojang,Mojang AB,33.15,2010
Wii Sports Resort for Wii,Wii,Nintendo,Nintendo EAD,33.13,2009
Pokemon Red / Green / Blue Version for GB,GB,Nintendo,Game Freak,31.38,1998
New Super Mario Bros. for DS,DS,Nintendo,Nintendo EAD,30.80,2006
New Super Mario Bros. Wii for Wii,Wii,Nintendo,Nintendo EAD,30.30,2009


In [164]:
%%sql 

select 
    count(*)
from
    game_sales
full join reviews
    using (game)
where 
    critic_score is null and --Found any records that had null values
    user_score is null;

 * postgresql:///games
1 rows affected.


count
31


In [168]:
--Selected year, average critic score, and number of games
--Joined datasets by game name
--Grouped by year
--Returned only records whose gameis greater than 4
--Ordered by critic score 

Select 
    year,
    round(avg(critic_score),2) as avg_critic_score,
    count(game_sales.game) as num_games 
from
    game_sales
inner join reviews
    using (game)   
group by 
        year
having
    count(game_sales.game) > 4   
order by 
    avg_critic_score desc
limit 10;
    

 * postgresql:///games
10 rows affected.


year,avg_critic_score,num_games
1998,9.32,10
2004,9.03,11
2002,8.99,9
1999,8.93,11
2001,8.82,13
2011,8.76,26
2016,8.67,13
2013,8.66,18
2008,8.63,20
2012,8.62,12


In [170]:
--Selected year, and avg crtic score
--Returns values excep those that are the same between each dataset

Select 
    year,
    avg_critic_score
from
    top_critic_years
except 
Select 
    year,
    avg_critic_score
from
    top_critic_years_more_than_four_games
order by 
    avg_critic_score desc;

 * postgresql:///games
6 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
2020,9.20
1993,9.10
1995,9.07
1982,9.00


In [172]:
%%sql 

-- Select the year, an average of user_score, and a count of games released in a given year, aliased and rounded
-- Included years with more than four reviewed games

Select 
    year, 
    round(avg(user_score),2) as avg_user_score,
    count(game) as num_games
from
    game_sales
inner join reviews
    using (game)
group by 
    year
having 
    count(game) > 4
order by
    avg_user_score desc
limit 10;

 * postgresql:///games
10 rows affected.


year,avg_user_score,num_games
1997,9.50,8
1998,9.40,10
2010,9.24,23
2009,9.18,20
2008,9.03,20
1996,9.00,5
2005,8.95,13
2006,8.95,16
2000,8.80,8
1999,8.80,11


In [ ]:
7

In [174]:
%%sql 

-- Selected the year results that appear on both tables
select 
    year 
from
    top_critic_years_more_than_four_games
inner join 
    top_user_years_more_than_four_games
    using (year)
group by year;

 * postgresql:///games
3 rows affected.


year
1998
2008
2002


In [ ]:
8

In [176]:
%%sql 

--Selected the year and number of games sold
--Placed the prvious query as a subquery to find what years are the same by year

Select 
    year,
    sum(games_sold) as total_games_sold
from
    game_sales
where 
    year in 
    (select 
        year 
    from
        top_critic_years_more_than_four_games
    inner join 
        top_user_years_more_than_four_games
        using (year)
    group by year 
    )
group by
    year
order by 
    total_games_sold desc;

 * postgresql:///games
3 rows affected.


year,total_games_sold
2008,175.07
1998,101.52
2002,58.67
